<a href="https://colab.research.google.com/github/Kamruzzamansust/Kamruzzamansust/blob/main/Text_Summarization1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

#what is sacebleu?


In [74]:
from transformers import pipeline ,set_seed
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset, load_metric
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm# what is it ?
import torch
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [75]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail','3.0.0')
print(f"Features in cnn_dailymail: {dataset['train'].column_names}")




Features in cnn_dailymail: ['article', 'highlights', 'id']


In [76]:
sample = dataset['train'][1]

len(sample['article']) , len(sample['highlights'])


(4051, 281)

In [77]:
sample

{'article': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less li

In [78]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [79]:
#text summerization pipline



In [80]:
sample_text = dataset['train'][1]['article'][:1000]
summaries={}

In [81]:
len(sample_text)

1000

In [117]:
#summerization Baseline

def baseline_summary_three_sent(text):
  return "\n".join(sent_tokenize(text)[:100])

In [83]:
summaries['baseline'] = baseline_summary_three_sent(sample_text)
summaries['baseline']

'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."\nHere, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court.\nMost often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies."\nHe says the arrests often result from confrontations with police.\nMentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely t

In [84]:
set_seed(42)

pipe= pipeline('text-generation',model = 'gpt2-medium')

gpt2_query = sample_text + "\nTL:DR:\n"

pipe_out = pipe(gpt2_query ,max_length= 512 )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [85]:
pipe_out # this is gpt2's output from my pipe

[{'generated_text': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they\'re ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won\'t do what they\'re told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and

In [86]:
generated_text = pipe_out[0]['generated_text']
start_index = generated_text.find(gpt2_query) + len(gpt2_query)
result_text = generated_text[start_index:]


print(result_text)


The Eighth Amendment of the U.S. Constitution protects citizens from unreasonable search & seizure. This is what Leifman said in a 2012 hearing before Florida's Florida Supreme Court when questioned by two men in their 50s at a detention facility for mentally ill inmates. When he started the hearing, the lawyers for these six men said these six were accused of being involved in what Leifman calls the "worst case of animal trafficking the U.S. has ever seen." They were not charged with committing a crime. But in his comments at the hearing, Leifman said that's how they would be treated if they were arrested and convicted. He said in 2011, there were about 100 cases per month in Florida where inmates charged with crimes were held after making no attempt to seek care or help. The facility's top official has acknowledged hundreds of these cases, the vast majority of which are mentally ill. All of these inmates face charges of failing to show up for their court dates or failing to sign or s

In [87]:
from nltk.tokenize import sent_tokenize

# Assuming result_text is the generated text
summaries['gpt2'] = "\n".join(sent_tokenize(result_text))


In [88]:
summaries['gpt2']

'The Eighth Amendment of the U.S. Constitution protects citizens from unreasonable search & seizure.\nThis is what Leifman said in a 2012 hearing before Florida\'s Florida Supreme Court when questioned by two men in their 50s at a detention facility for mentally ill inmates.\nWhen he started the hearing, the lawyers for these six men said these six were accused of being involved in what Leifman calls the "worst case of animal trafficking the U.S. has ever seen."\nThey were not charged with committing a crime.\nBut in his comments at the hearing, Leifman said that\'s how they would be treated if they were arrested and convicted.\nHe said in 2011, there were about 100 cases per month in Florida where inmates charged with crimes were held after making no attempt to seek care or help.\nThe facility\'s top official has acknowledged hundreds of these cases, the vast majority of which are mentally ill. All of these inmates face charges of failing to show up for their court dates or failing to

**T5**

In [89]:
''' T5 is well known transformers which is used for text generation ,
text summarization,Q&A ,Classification task ,machine translation
how it is different from bert?


'''

' T5 is well known transformers which is used for text generation ,\ntext summarization,Q&A ,Classification task ,machine translation\nhow it is different from bert?\n\n\n'

In [90]:
#note : using hugging face pipline we can directly use T5

In [91]:
pipe = pipeline( 'summarization' , model = 't5-small')

pipe_out2 = pipe(sample_text)

In [92]:
pipe_out2

[{'summary_text': "inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."}]

In [93]:
pipe_out2[0]['summary_text']

"inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court . most often, they face drug charges or charges of assaulting an officer . mentally ill people become more paranoid, delusional, and less likely to follow dir ."

In [94]:
summaries['t5']='n'.join(sent_tokenize(pipe_out2[0]['summary_text']))

In [95]:
summaries['t5']

"inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court .nmost often, they face drug charges or charges of assaulting an officer .nmentally ill people become more paranoid, delusional, and less likely to follow dir ."

In [96]:
#BART
'''
BART, introduced by Facebook in 2019, is
designed for both denoising autoencoder pretraining and
sequence-to-sequence (seq2seq) downstream tasks. It can be used for tasks
like text summarization, language generation, and translation.

For downstream tasks, it is used as a seq2seq model with a
pretrained encoder-decoder architecture.
BART has shown effectiveness in various generative tasks, particularly
in abstractive text summarization.

BERT was introduced by Google in 2018, and its primary goal is to
pretrain deep bidirectional representations from unlabeled text.
It learns contextualized word representations by considering both
the left and
right context of each word in a sentence.

 BERT uses a masked language model (MLM) objective(it is like fill in the blank)
 during pretraining. It randomly masks out some words in a
 sentence, and the model is trained to predict those masked words
 based on the context of the surrounding words.
 BERT has been influential in various natural language processing (NLP)
 tasks, and fine-tuned versions of BERT have been used for tasks like
 text classification,
 named entity recognition, and question answering


'''


'\nBART, introduced by Facebook in 2019, is\ndesigned for both denoising autoencoder pretraining and\nsequence-to-sequence (seq2seq) downstream tasks. It can be used for tasks\nlike text summarization, language generation, and translation.\n\nFor downstream tasks, it is used as a seq2seq model with a\npretrained encoder-decoder architecture.\nBART has shown effectiveness in various generative tasks, particularly\nin abstractive text summarization.\n\nBERT was introduced by Google in 2018, and its primary goal is to\npretrain deep bidirectional representations from unlabeled text.\nIt learns contextualized word representations by considering both\nthe left and\nright context of each word in a sentence.\n\n BERT uses a masked language model (MLM) objective(it is like fill in the blank)\n during pretraining. It randomly masks out some words in a\n sentence, and the model is trained to predict those masked words\n based on the context of the surrounding words.\n BERT has been influential i

In [97]:
pipe = pipeline('summarization' , model = 'facebook/bart-large-cnn')

pipe_out_3 = pipe(sample_text)

In [98]:
pipe_out_3

[{'summary_text': 'Miami-Dade pretrial detention facility is dubbed the "forgotten floor" Here, inmates with the most severe mental illnesses are incarcerated. Most often, they face drug charges or charges of assaulting an officer. Judge Steven Leifman says the arrests often result from confrontations with police.'}]

In [99]:
summaries['bart'] = '\n'.join(sent_tokenize(pipe_out_3[0]['summary_text']))

In [100]:
type(summaries['bart'])

str

In [101]:
#PEGASUS
'''
PEGASUS is a model in natural language processing (NLP)
developed by Google. It is specifically designed for abstractive
text summarization, which involves generating a concise and coherent
summary of a given input document. PEGASUS was introduced in the paper
titled "PEGASUS: Pre-training with Extracted Gap-sentences for
Abstractive Summarization," presented at NeurIPS 2019.
The model has demonstrated strong performance on various
summarization benchmarks and has been used for tasks such as
news article
summarization and document summarization.
The primary application of PEGASUS is abstractive summarization,
where the model generates a summary that may not be a verbatim extract
from the input document but captures its essential meaning. This is in
contrast to extractive summarization, which selects and combines
existing sentences from the document

The  main advantage of pegasus is that it deals with misiing sentences with self supervision

'''

'\nPEGASUS is a model in natural language processing (NLP)\ndeveloped by Google. It is specifically designed for abstractive\ntext summarization, which involves generating a concise and coherent\nsummary of a given input document. PEGASUS was introduced in the paper\ntitled "PEGASUS: Pre-training with Extracted Gap-sentences for\nAbstractive Summarization," presented at NeurIPS 2019.\nThe model has demonstrated strong performance on various\nsummarization benchmarks and has been used for tasks such as\nnews article\nsummarization and document summarization.\nThe primary application of PEGASUS is abstractive summarization,\nwhere the model generates a summary that may not be a verbatim extract\nfrom the input document but captures its essential meaning. This is in\ncontrast to extractive summarization, which selects and combines\nexisting sentences from the document\n\nThe  main advantage of pegasus is that it deals with misiing sentences with self supervision\n\n'

In [102]:
pipe = pipeline('summarization' , model ="google/pegasus-cnn_dailymail" )
pipe_out_4 = pipe(sample_text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [103]:
pipe_out_4

[{'summary_text': 'Mentally ill inmates are housed on the "forgotten floor" of a Miami jail .<n>Judge Steven Leifman says the charges are usually "avoidable felonies"<n>He says the arrests often result from confrontations with police .<n>Mentally ill people often won\'t do what they\'re told when police arrive on the scene .'}]

In [104]:
summaries['pegasus'] = "\n".join(sent_tokenize(pipe_out_4[0]['summary_text']))

In [107]:
summaries['pegasus']=summaries['pegasus'].replace(" .<n>",'\n')

In [108]:
type(summaries['pegasus'])

str

In [109]:
#checking evaluation metric Sacrebleu


from datasets import load_metric

bleu_metric = load_metric('sacrebleu')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [110]:
bleu_metric.add (prediction =[summaries["pegasus"]], reference = [dataset['train'][1]['highlights'] ])

results = bleu_metric.compute(smooth_method = 'floor',smooth_value = 0)
results['precision'] = [np.round(p,2) for p in results['precisions']]
pd.DataFrame.from_dict(results,orient = 'index' , columns = ['value'])



,value
score,19.328777
counts,"[27, 14, 10, 6]"
totals,"[65, 64, 63, 62]"
precisions,"[41.53846153846154, 21.875, 15.873015873015873..."
bp,1.0
sys_len,65
ref_len,57
precision,"[41.54, 21.88, 15.87, 9.68]"


In [111]:
#rouge

In [112]:
# why we need rouge over bleu?

'''

ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
and BLEU (Bilingual Evaluation Understudy) are both evaluation
metrics used in natural language processing tasks, particularly in the
context of text summarization and machine translation. However, they have
different focuses and capture different
aspects of the quality of generated text.

ROUGE: ROUGE is recall-oriented, which means it places more emphasis
on the overlap of content between the reference (ground truth) and the
generated text. It is designed to measure how well the
generated text covers the information present in the reference.

BLEU: BLEU, on the other hand, is precision-oriented.
It primarily measures how many of the generated n-grams (word sequences)
match those in the reference. While precision is essential,
it may not be sufficient for tasks where recall is crucial.

'''


'\n\nROUGE (Recall-Oriented Understudy for Gisting Evaluation) \nand BLEU (Bilingual Evaluation Understudy) are both evaluation \nmetrics used in natural language processing tasks, particularly in the \ncontext of text summarization and machine translation. However, they have \ndifferent focuses and capture different \naspects of the quality of generated text.\n\nROUGE: ROUGE is recall-oriented, which means it places more emphasis \non the overlap of content between the reference (ground truth) and the \ngenerated text. It is designed to measure how well the \ngenerated text covers the information present in the reference.\n\nBLEU: BLEU, on the other hand, is precision-oriented. \nIt primarily measures how many of the generated n-grams (word sequences) \nmatch those in the reference. While precision is essential, \nit may not be sufficient for tasks where recall is crucial.\n\n'

In [113]:
#load rouge metric from hugging face


!pip install rouge_score

In [114]:
from datasets import load_metric

rouge_metric = load_metric("rouge")


# plz google to know about ROUHE-N and ROUGE-L

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [115]:
from datasets import load_metric
import pandas as pd

# Load the ROUGE metric
rouge_metric = load_metric("rouge")

rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

reference = dataset['train'][1]['highlights']
#summaries = {'model1': 'Generated summary 1', 'model2': 'Generated summary 2'}  # Replace with your actual summaries

records = []

for model_name, hypothesis in summaries.items():
    rouge_metric.add(prediction=hypothesis, reference=reference)
    score = rouge_metric.compute()
    rouge_dict = {rn: score[rn].mid.fmeasure for rn in rouge_names}
    records.append(rouge_dict)

df_results = pd.DataFrame.from_records(records, index=summaries.keys())
print(df_results)


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


            rouge1    rouge2    rougeL  rougeLsum
baseline  0.312796  0.133971  0.170616   0.284360
gpt2      0.140127  0.019231  0.082803   0.133758
t5        0.175824  0.000000  0.131868   0.153846
bart      0.365591  0.131868  0.215054   0.322581
pegasus   0.500000  0.244898  0.360000   0.460000


In [115]:
#evaluation on the test set